
# 📊 Speedtest Telemetry — Exploratory Analysis

This notebook loads and analyzes your `speedtest.csv` produced by the speedtest-logger script.

**Expected columns**
- `timestamp` (string or ISO datetime)
- `download_mbps`, `upload_mbps`
- `ping_ms`, `jitter_ms`
- `packet_loss` (percentage or fraction)
- `server_name`, `server_id`
- `isp`
- `status` (ok/error)
- `error` (optional message)

> Tip: If your CSV has different headers, set `COLUMN_MAP` below to rename them on load.


In [ ]:

# ---- Config ----
from pathlib import Path

# Default path assumes this notebook lives in: <repo_root>/notebooks/
# and CSV is in: <repo_root>/data/speedtest.csv
DEFAULT_CSV = (Path.cwd().parent / "data" / "speedtest.csv")

# Alternatively, override with an absolute path:
# DEFAULT_CSV = Path("/home/youruser/speedtest-logs/data/speedtest.csv")

# Map your CSV headers to canonical names if needed
COLUMN_MAP = {
    # 'dl': 'download_mbps',
    # 'ul': 'upload_mbps',
    # 'latency_ms': 'ping_ms',
}

DEFAULT_CSV


In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 120)


In [ ]:

csv_path = Path(DEFAULT_CSV)
assert csv_path.exists(), f"CSV not found: {csv_path}"

df = pd.read_csv(csv_path)
if COLUMN_MAP:
    df = df.rename(columns=COLUMN_MAP)

# Normalize column names
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]

# Parse time
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df = df.sort_values('timestamp')
else:
    raise ValueError("Expected a 'timestamp' column.")

# Ensure numeric columns
for col in ['download_mbps','upload_mbps','ping_ms','jitter_ms','packet_loss']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Clean obvious outliers (optional thresholds; tweak as needed)
df['packet_loss'] = df['packet_loss'].clip(lower=0) if 'packet_loss' in df.columns else np.nan

# Derive time features
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['dow']  = df['timestamp'].dt.day_name()

df.head()


## 📐 Basic Summary

In [ ]:

summary = {}
for col in ['download_mbps','upload_mbps','ping_ms','jitter_ms','packet_loss']:
    if col in df.columns:
        summary[col] = df[col].describe(percentiles=[0.1,0.25,0.5,0.75,0.9,0.95]).to_dict()
pd.DataFrame(summary)


## 🕒 Time Series

In [ ]:

def ts_plot(series, ylabel):
    plt.figure(figsize=(10,4))
    plt.plot(df['timestamp'], series)
    plt.title(ylabel + ' over time')
    plt.xlabel('timestamp')
    plt.ylabel(ylabel)
    plt.tight_layout()
    plt.show()

if 'download_mbps' in df.columns:
    ts_plot(df['download_mbps'], 'Download (Mbps)')

if 'upload_mbps' in df.columns:
    ts_plot(df['upload_mbps'], 'Upload (Mbps)')

if 'ping_ms' in df.columns:
    ts_plot(df['ping_ms'], 'Latency (ms)')

if 'jitter_ms' in df.columns:
    ts_plot(df['jitter_ms'], 'Jitter (ms)')

if 'packet_loss' in df.columns:
    ts_plot(df['packet_loss'], 'Packet Loss')


## 📈 Rolling Averages (noise reduction)

In [ ]:

# Use a 5-sample rolling window; adjust to your cadence
if 'download_mbps' in df.columns:
    df['dl_roll'] = df['download_mbps'].rolling(window=5, min_periods=1).mean()
    plt.figure(figsize=(10,4))
    plt.plot(df['timestamp'], df['download_mbps'], alpha=0.4, label='raw')
    plt.plot(df['timestamp'], df['dl_roll'], label='rolling(5)')
    plt.title('Download (Mbps): raw vs rolling mean')
    plt.xlabel('timestamp')
    plt.ylabel('Mbps')
    plt.legend()
    plt.tight_layout()
    plt.show()

if 'upload_mbps' in df.columns:
    df['ul_roll'] = df['upload_mbps'].rolling(window=5, min_periods=1).mean()
    plt.figure(figsize=(10,4))
    plt.plot(df['timestamp'], df['upload_mbps'], alpha=0.4, label='raw')
    plt.plot(df['timestamp'], df['ul_roll'], label='rolling(5)')
    plt.title('Upload (Mbps): raw vs rolling mean')
    plt.xlabel('timestamp')
    plt.ylabel('Mbps')
    plt.legend()
    plt.tight_layout()
    plt.show()


## ⏰ Aggregations by Hour & Day

In [ ]:

agg_hour = df.groupby('hour').agg(
    dl=('download_mbps','mean'),
    ul=('upload_mbps','mean'),
    ping=('ping_ms','mean'),
    jitter=('jitter_ms','mean'),
    loss=('packet_loss','mean')
)
display(agg_hour)

agg_day = df.groupby('date').agg(
    dl=('download_mbps','mean'),
    ul=('upload_mbps','mean'),
    ping=('ping_ms','mean'),
    jitter=('jitter_ms','mean'),
    loss=('packet_loss','mean')
)
display(agg_day.tail())


## 🖥️ Server/ISP Analysis

In [ ]:

if 'server_name' in df.columns:
    server_perf = df.groupby('server_name').agg(
        tests=('server_name','count'),
        dl=('download_mbps','median'),
        ul=('upload_mbps','median'),
        ping=('ping_ms','median'),
        loss=('packet_loss','median')
    ).sort_values('tests', ascending=False)
    display(server_perf.head(10))

if 'isp' in df.columns:
    isp_perf = df.groupby('isp').agg(
        tests=('isp','count'),
        dl=('download_mbps','median'),
        ul=('upload_mbps','median'),
        ping=('ping_ms','median'),
        loss=('packet_loss','median')
    ).sort_values('tests', ascending=False)
    display(isp_perf)


## 🚨 Outage & Degradation Detection

In [ ]:

DL_MIN = 10   # Mbps thresholds; tune for your expected service
UL_MIN = 5
MAX_PING = 200  # ms
MAX_LOSS = 5.0  # percent

alerts = []

if 'download_mbps' in df.columns:
    alerts.append(df['download_mbps'] < DL_MIN)
if 'upload_mbps' in df.columns:
    alerts.append(df['upload_mbps'] < UL_MIN)
if 'ping_ms' in df.columns:
    alerts.append(df['ping_ms'] > MAX_PING)
if 'packet_loss' in df.columns:
    alerts.append(df['packet_loss'] > MAX_LOSS)

if alerts:
    import numpy as np
    bad = np.logical_or.reduce(alerts)
    cols = [c for c in ['timestamp','download_mbps','upload_mbps','ping_ms','jitter_ms','packet_loss','server_name','server_id','status','error'] if c in df.columns]
    issues = df.loc[bad, cols].copy()
    display(issues.tail(20))
else:
    print("No alert conditions defined or columns missing.")


## 💾 Save a cleaned/enhanced CSV

In [ ]:

out_path = (Path.cwd().parent / 'data' / 'speedtest_cleaned.csv')
save_cols = [c for c in ['timestamp','download_mbps','upload_mbps','ping_ms','jitter_ms','packet_loss','server_name','server_id','isp','status','error','dl_roll','ul_roll','date','hour','dow'] if c in df.columns]
df.to_csv(out_path, index=False, columns=save_cols)
out_path
